# 波士顿房价预测案例——特征工程

通过数据探索，我们了解了数据集的特点。接下来进行特征工程：对原始特征做必要的数据预处理和特征编码，使得变换后的特征符合模型要求。

## 1、导入必要的工具包

In [1]:
import numpy as np  # 矩阵操作
import pandas as pd # SQL数据处理

## 2. 读取数据
该数据集很简单，可以直接送入回归模型；
一般而言，我们通常先对原始特征进行必要的特征编码和处理（特征工程），编码后的特征再送入模型。

In [2]:
# path to where the data lies
#dpath = './data/'
df = pd.read_csv("boston_housing.csv")

#通过观察前5行，了解数据每列（特征）的概况
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15,396.90,4.98,24.0
1,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17,396.90,9.14,21.6
2,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17,392.83,4.03,34.7
3,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18,394.63,2.94,33.4
4,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18,396.90,5.33,36.2


###  数据基本信息
样本数目、特征维数
每个特征的类型、空值样本的数目、数据类型

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null int64
INDUS      506 non-null float64
CHAS       506 non-null int64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null int64
TAX        506 non-null int64
PTRATIO    506 non-null int64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(9), int64(5)
memory usage: 55.4 KB


## 3. 特征工程

### 3.1. 数据去噪

In [4]:
# 删除y大于等于50的样本（保留小于50的样本）
#df = df[df.MEDV < 50]

#输出样本数和特征维数
#print(df.shape)

去掉了16个样本

### 3.2 数据分离
从原始数据中分离输入特征X和标签y

In [17]:
# 从原始数据中分离输入特征x和输出y
y = df['MEDV']
X = df.drop('MEDV', axis = 1)

# 尝试对y（房屋价格）做log变换，对log变换后的价格进行估计
log_y = np.log1p(y)
log_y = pd.DataFrame(log_y)
print log_y

         MEDV
0    3.218876
1    3.117950
2    3.575151
3    3.538057
4    3.616309
5    3.391147
6    3.173878
7    3.335770
8    2.862201
9    2.990720
10   2.772589
11   2.990720
12   3.122365
13   3.063391
14   2.954910
15   3.039749
16   3.182212
17   2.917771
18   3.054001
19   2.954910
20   2.681022
21   3.025291
22   2.785011
23   2.740840
24   2.809403
25   2.701361
26   2.867899
27   2.760010
28   2.965273
29   3.091042
..        ...
476  2.873565
477  2.564949
478  2.747271
479  3.109061
480  3.178054
481  3.206803
482  3.258097
483  3.126761
484  3.072693
485  3.100092
486  3.000720
487  3.072693
488  2.785011
489  2.079442
490  2.208274
491  2.681022
492  3.049273
493  3.126761
494  3.238678
495  3.182212
496  3.030134
497  2.960105
498  3.100092
499  2.917771
500  2.879198
501  3.152736
502  3.072693
503  3.214868
504  3.135494
505  2.557227

[506 rows x 1 columns]


### 3.3 离散型特征编码

离散特征可以通过独热编码（one-hot encode），将原来有K种取值的离散型特征变成K维0-1编码特征，这K维特征中只有一个是1（独热），其余维均为0.
独热编码可以用pandas的get_dummies方法（哑编码）或者Scikit-Learn中的OneHotEncoder类来实现。

get_dummies方法要求输入特征的类型是非数值型（"object"）；
而OneHotEncoder要求输入是整数。如果是字符串要先用LabelEncoder变成整数。（但LabelEncoder输出是一维（1D）数组，而OneHotEncoder要求输出是2D数组，需要在二者之间进行格式转换）。

另外如果训练数据和测试数据不能同时获得的话，需要用OneHotEncoder，用训练集训练编码器，然后对训练集和测试集进行编码；
而get_dummies是依赖于DataFrame，只适用于一个数据集情况。

更多学习，推荐阅读：[scikit-learn] 特征二值化编码函数的一些坑
https://ask.hellobi.com/blog/DataMiner/4897

In [6]:
# RAD的含义是距离高速公路的便利指数。虽然给的数值是数值型，但实际是索引，可换成离散特征/类别型特征编码试试。
X["RAD"].astype("object")
X_cat = X["RAD"]
X_cat = pd.get_dummies(X_cat, prefix="RAD")

X = X.drop("RAD", axis = 1)

#特征名称，用于保存特征工程结果
feat_names = X.columns

In [7]:
X_cat.head()

,RAD_1,RAD_2,RAD_3,RAD_4,RAD_5,RAD_6,RAD_7,RAD_8,RAD_24
0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0


###  3.4. 数值型特征预处理
scikit learn中提供的数据预处理功能：
http://scikit-learn.org/stable/modules/preprocessing.html

Boston房价数据集特征均为数值型特征。在数据探索阶段发现，发现各特征差异较大，需要进行数据标准化预处理。
标准化的目的在于避免原始特征值差异过大，导致训练得到的参数权重单位不一致，无法比较各特征的重要性。
另外，一些优化算法（如随机梯度下降及其改进版本）只在各特征尺度差不多的情况下才能保证收敛。

#### 数值特征标准化

In [18]:
# 数据标准化
from sklearn.preprocessing import StandardScaler

# 分别初始化对特征和目标值的标准化器
ss_X = StandardScaler()
ss_y = StandardScaler()

ss_log_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
# 对训练数据，先调用fit方法训练模型，得到模型参数；然后对训练数据和测试数据进行transform
X = ss_X.fit_transform(X)

#对y做标准化不是必须
#对y标准化的好处是不同问题的w差异不太大，同时正则参数的范围也有限
y = ss_y.fit_transform(y.reshape(-1, 1))
log_y = ss_y.fit_transform(log_y.reshape(-1, 1))

AttributeError: 'Series' object has no attribute 'reshape'

## 数据特征最小最大缩放化

In [16]:
# 数据最小最大缩放化
from sklearn.preprocessing import MinMaxScaler

# 分别初始化对特征和目标值的最小最大缩放器
ss_X = MinMaxScaler()
ss_y = MinMaxScaler()

ss_log_y = MinMaxScaler()

# 分别对训练和测试数据的特征以及目标值进行最小最大缩放处理
# 对训练数据，先调用fit方法训练模型，得到模型参数；然后对训练数据和测试数据进行transform
X = ss_X.fit_transform(X)


y = ss_y.fit_transform(y.reshape(1,-1))
log_y = ss_y.fit_transform(log_y.reshape(1,-1))

AttributeError: 'Series' object has no attribute 'reshape'

## 4. 保存特征工程的结果到文件，供机器学习模型使用

In [ ]:
fe_data = pd.DataFrame(data = X, columns = feat_names, index = df.index)
fe_data = pd.concat([fe_data, X_cat], axis = 1, ignore_index=False)

#加上标签y
fe_data["MEDV"] = y
fe_data["log_MEDV"] = log_y

#保存结果到文件
fe_data.to_csv('FE_boston_housing.csv', index=False)

In [ ]:
fe_data.head()

In [ ]:
fe_data.info()